In [1]:
import pandas as pd
import copy
import cvxpy as cp
import numpy as np
from scipy import stats
import json

In [2]:
data = pd.read_csv('car.data', names=['buying', 'maint', 'doors', 'persons', 'lug_bool', 'safety', 'class'])
data['y'] = data['class']
data['y'][data['class'] == 'unacc'] = 0
data['y'][data['class'] != 'unacc'] = 1

In [3]:
data.head(5)

,buying,maint,doors,persons,lug_bool,safety,class,y
0,vhigh,vhigh,2,2,small,low,unacc,0
1,vhigh,vhigh,2,2,small,med,unacc,0
2,vhigh,vhigh,2,2,small,high,unacc,0
3,vhigh,vhigh,2,2,med,low,unacc,0
4,vhigh,vhigh,2,2,med,med,unacc,0


In [4]:
X = pd.get_dummies(data[['buying', 'maint', 'doors', 'persons', 'lug_bool', 'safety']]).to_numpy()
y = data['y'].to_numpy().astype('int')

In [5]:
X.shape

(1728, 21)

In [6]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(penalty='l2')
logreg.fit(X, y)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
y_pred = logreg.predict(X)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X, y)))

Accuracy of logistic regression classifier on test set: 0.95


In [8]:
prefer_params = 1 / (1 + np.exp( - (np.matmul(X, np.transpose(logreg.coef_)) + logreg.intercept_)))

In [9]:
stats.describe(prefer_params)

DescribeResult(nobs=1728, minmax=(array([9.53546617e-10]), array([0.9988827])), mean=array([0.3005815]), variance=array([0.15438713]), skewness=array([0.81649976]), kurtosis=array([-1.10981301]))

In [10]:
params = dict()
params['preference_params'] = [1] + list(prefer_params.flatten())
params['revenues'] = [0]
params['revenues'].extend(list(np.random.uniform(0.1, 1, X.shape[0])))

with open('params.json', 'w') as fp:
  json.dump(params, fp)